In [1]:
import pandas as pd

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'
    # path = '/Users/oliverjcarter/Documents/GitHub/f1-analytics/'

In [18]:
races = pd.read_csv(path+'data/races.csv')
qualifying = pd.read_csv(path+'data/qualifying.csv')
constructors = pd.read_csv(path+'data/constructor_standings.csv')
drivers = pd.read_csv(path+'data/driver_standings.csv')
results = pd.read_csv(path+'data/results.csv')

### Redundant until new weather collection method is implemented
# weather = pd.read_csv(path+'data/weather.csv')

In [2]:
laps1 = pd.read_csv(path+'data/laps-2014-2017.csv')
laps2 = pd.read_csv(path+'data/laps-2017-22.csv')

In [8]:
laps = pd.merge(laps1, laps2, how='left', on=['season', 'round', 'driver', 'position', 'lap', 'time'])

,season,round,lap,driver,position,time
0,2014,1,1,rosberg,1,1:42.038
1,2014,1,1,ricciardo,2,1:43.549
2,2014,1,1,kevin_magnussen,3,1:45.402
3,2014,1,1,hamilton,4,1:46.128
4,2014,1,1,hulkenberg,5,1:46.986


In [11]:
duplicates = laps[laps.duplicated()]

,season,round,lap,driver,position,time


In [ ]:
laps.to_csv(path+'data/laps.csv', index=False)

In [28]:
results.query('grid == 0')

,season,round,circuit_id,driver,date_of_birth,nationality,constructor,grid,time,status,points,podium,fastest_lap
424,2015,1,albert_park,bottas,1989-08-28,Finnish,williams,0,NaN,Withdrew,0.0,18,NaN
444,2015,2,sepang,stevens,1991-06-28,British,manor,0,NaN,Fuel system,0.0,20,NaN
514,2015,6,monaco,sainz,1994-09-01,Spanish,toro_rosso,0,6583476.0,Finished,1.0,10,1:19.816
911,2016,6,monaco,nasr,1992-08-21,Brazilian,sauber,0,NaN,Retired,0.0,17,1:21.889
912,2016,6,monaco,max_verstappen,1997-09-30,Dutch,red_bull,0,NaN,Accident,0.0,18,1:26.563
980,2016,9,red_bull_ring,massa,1981-04-25,Brazilian,williams,0,NaN,Brakes,0.0,20,1:09.899
982,2016,9,red_bull_ring,kvyat,1994-04-26,Russian,toro_rosso,0,NaN,Retired,0.0,22,1:18.302
1263,2017,1,albert_park,ricciardo,1989-07-01,Australian,red_bull,0,NaN,Power Unit,0.0,17,1:29.447
1546,2017,15,sepang,raikkonen,1979-10-17,Finnish,ferrari,0,NaN,Battery,0.0,20,NaN
1846,2018,10,silverstone,brendon_hartley,1989-11-10,New Zealander,toro_rosso,0,NaN,Power Unit,0.0,20,NaN


In [19]:
def lookup (df, team, points):
    df['lookup1'] = df.season.astype(str) + df[team] + df['round'].astype(str)
    df['lookup2'] = df.season.astype(str) + df[team] + (df['round']-1).astype(str)
    new_df = df.merge(df[['lookup1', points]], how = 'left', left_on='lookup2',right_on='lookup1')
    new_df.drop(['lookup1_x', 'lookup2', 'lookup1_y'], axis = 1, inplace = True)
    new_df.rename(columns = {points+'_x': points+'_after_race', points+'_y': points}, inplace = True)
    new_df[points].fillna(0, inplace = True)
    return new_df

In [20]:
drivers = lookup(drivers, 'driver', 'driver_points')
drivers = lookup(drivers, 'driver', 'driver_wins')
drivers = lookup(drivers, 'driver', 'driver_standings_pos')

drivers.drop(['driver_points_after_race', 'driver_wins_after_race', 'driver_standings_pos_after_race'], axis=1, inplace=True)

In [21]:
constructors = lookup(constructors, 'constructor', 'constructor_points')
constructors = lookup(constructors, 'constructor', 'constructor_wins')
constructors = lookup(constructors, 'constructor', 'constructor_standings_pos')

constructors.drop(['constructor_points_after_race', 'constructor_wins_after_race', 'constructor_standings_pos_after_race'], axis=1, inplace=True)

In [22]:
df1 = pd.merge(races, results, how='inner', on=['season', 'round', 'circuit_id']).drop(['url','points', 'status', 'time'], axis = 1)
df2 = pd.merge(df1, drivers, how='left', on=['season', 'round', 'driver']) 
df3 = pd.merge(df2, constructors, how='left', on=['season', 'round', 'constructor'])

merged_df = pd.merge(df3, qualifying, how='inner', on=['season', 'round', 'circuit_id', 'driver']).drop('position', axis=1)

merged_df.shape

(3671, 23)

In [23]:
merged_df.columns

Index(['season', 'round', 'circuit_id', 'country', 'lat', 'long', 'date',
       'driver', 'date_of_birth', 'nationality', 'constructor', 'grid',
       'podium', 'fastest_lap', 'driver_points', 'driver_wins',
       'driver_standings_pos', 'constructor_points', 'constructor_wins',
       'constructor_standings_pos', 'fastest_time', 'stage', 'q_delta'],
      dtype='object')

In [24]:
merged_df.isna().any()

season                       False
round                        False
circuit_id                   False
country                      False
lat                          False
long                         False
date                         False
driver                       False
date_of_birth                False
nationality                  False
constructor                  False
grid                         False
podium                       False
fastest_lap                   True
driver_points                False
driver_wins                  False
driver_standings_pos         False
constructor_points           False
constructor_wins             False
constructor_standings_pos    False
fastest_time                  True
stage                         True
q_delta                      False
dtype: bool

In [25]:
# fill/drop nulls
### Data is being lossed here

for col in ['driver_points', 'driver_wins', 'driver_standings_pos', 'constructor_points', 
            'constructor_wins', 'constructor_standings_pos']:
    merged_df[col].fillna(0, inplace=True)
    merged_df[col] = merged_df[col].map(lambda x: int(x))
    
merged_df.dropna(inplace=True)

merged_df.shape

(3462, 23)

In [26]:
from dateutil.relativedelta import *

merged_df['date_of_birth'] = pd.to_datetime(merged_df.date_of_birth)
merged_df['driver_age'] = merged_df.apply(lambda x: relativedelta(pd.to_datetime(x['date']), x['date_of_birth']).years, axis=1)
merged_df.drop(['date_of_birth'], axis=1, inplace = True)

In [27]:
merged_df.to_csv(path+'data/merged.csv', index=False)